In [1]:
import importlib
import extract
import graph
importlib.reload(extract)
importlib.reload(graph)
import time
import matplotlib.pyplot as plt

In [20]:
start = time.time()
data = extract.Data("./data/CIS_enrollment.csv", ["Computer Science"])
end = time.time()
print(end - start)

1.0154800415039062


# Data model test

In [21]:
len(data.courses),len(data.students)

(2946, 3194)

In [22]:
print(list(data.students.values())[0])

{'id': '321398', 'major': 'Computer Science', 'courses': ['Physics I: Mechanics & Heat', 'Advanced Computer Architecture', 'Complex Digital ASIC Design', 'Intro to Microelectronics', 'MicroElectro Mechanical System', 'Obj-Oriented Prog & Data Struc', 'Computer Graphics I', 'Differential Equations Engrs', 'Intro to Scientific Comp', 'Fiber & Integrated Optics', 'Lasers and Optoelectronics', 'Advanced Volleyball', 'Engineering Seminar', 'Intro Microeconomics', 'Multivariable Calculus Engrs', 'Digital VLSI Design', 'Embedded Systems', 'Physics II: Electromagnetism', 'Engineering Communications', 'Datacenter Networks & Services', 'Advanced Digital VLSI', 'Intro Computing Using Java', 'Linear Algebra for Engineers', 'Intro to Entrep & Engr Enterpr', 'Archery', 'Beginning Swim', 'Intro to Circ Elec & Comp Engr', 'Phys III-Osc Waves & Quan Phys', 'Short Course in MATLAB', 'Marketing', 'Cooperative Workshop MATH 1920', 'Advanced Computer Architecture', 'Signals and Information', 'Finance', 'In

In [23]:
print(list(data.courses["CS1110"].students)[0])

{'id': '2527378', 'major': 'Computer Science', 'courses': ['Biological Statistics II', 'Living Routines', 'Machine Learning Data Science', 'Introduction To Linguistics', 'Programming Practicum', 'Intro to Natural Lang Process', 'Adv Game Architecture', 'Introduction to Psychology', 'Italian II', 'Machine Learning', 'Great Discoveries/Archaeology', 'Industrial Data & Sys. Analy.', 'Calculus II', 'C++ Programming', 'Operating Systems', 'Statistical Data Mining I', 'Boxing', 'Green World/Blue Planet', 'Networks', 'Intro Computing Using Java', 'Linear Algebra for Engineers', 'Calculus I', 'Biological Statistics I', 'Introduction to Wines', 'Data Struct & Functional Progr', 'Probability Models & Inference', 'UNIX Tools and Scripting', 'Our Solar System', 'Systems Programming', 'Communication Adv Game Dev', 'Prac in A I', 'Intro Analysis of Algorithms', 'Discrete Structures', 'Obj-Oriented Prog & Data Struc', 'Introduction to China'], 'terms': [('CS4152', (2018, 0)), ('HADM4300', (2018, 0)),

# Graph model test

In [39]:
cog = graph.Graph()
postg = graph.Graph()

In [40]:
def build_graph(g, n = 0):
    start = time.time()
    c = 1
    for cname1 in data.courses:
        if c % 250 == 0:
            print(c)
        for cname2 in data.courses:
            if cname2 == cname1:
                continue
            course = data.courses[cname1]
            count = 0
            for s in course.students:
                if cname2 in s.term_numbers and s.term_numbers[cname2] == (s.term_numbers[cname1] + n):
                    count += 1
            if count > 0:
                g.addEdge(cname1, cname2, count)
        c += 1
    end = time.time()
    print(end - start)

In [41]:
build_graph(cog)
build_graph(postg, 1)

250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
54.47909903526306
250
500
750
1000
1250
1500
1750
2000
2250
2500
2750
38.64127016067505


In [47]:
print(len(postg.edges))
count = 0
for k in postg.edges:
    k1, k2 = k
    if k in postg.edges and (k2,k1) in postg.edges:
        w1, w2 = postg.edges[k].weight, postg.edges[(k2, k1)].weight
        if w1/w2 > 2.0:
            count += 1
print(count)

111195
6010


In [49]:
print(postg.edges[("CS2110","CS3110")])
print(postg.edges[("CS2110","CS3410")])
print(cog.edges[("CS3110","CS3410")])
print(cog.edges[("CS3110","CS2800")])
print(postg.edges[("CS2800","CS3110")])

(CS2110,CS3110,491)
(CS2110,CS3410,354)
(CS3110,CS3410,43)
(CS3110,CS2800,388)
(CS2800,CS3110,958)


In [27]:
# g.export_json("data/cs_co_30",30)

In [28]:
# g.export_graph("test4",300)

In [36]:
x = [min(e.weight, 100) for e in g.edges]
num_bins = 25
n, bins, patches = plt.hist(x, num_bins, facecolor='blue', alpha=0.5)
plt.xlabel("edge weight")
plt.ylabel("frequency")
plt.title("distribution of edge weights (capped at 100)")
plt.show()

AttributeError: 'tuple' object has no attribute 'weight'

In [134]:
edges_dict = {}
for e in g.edges:
    edges_dict[(e.src, e.dest)] = e.weight
print(len(edges_dict))

159731


In [135]:
for k in list(edges_dict.keys()):
    k1, k2 = k
    if k in edges_dict and (k2,k1) in edges_dict:
        w1, w2 = edges_dict[k], edges_dict[(k2, k1)]
        if w1/w2 > 4:
            del edges_dict[(k2, k1)]
        elif w2/w1 > 4:
            del edges_dict[k]
print(len(edges_dict))

156744


In [139]:
for k in list(edges_dict.keys()):
    if edges_dict[k] < 5:
        del edges_dict[k]
print(len(edges_dict))    

14225


In [141]:
n = set()
for k in list(edges_dict.keys()):
    n.add(k[0])
    n.add(k[1])
print(len(n))

922


In [142]:
len(g.nodes)

3625

In [20]:
len(set([x.src for x in g.filterEdges(lambda e: e.weight > 30)]+[x.dest for x in g.filterEdges(lambda e: e.weight > 30)]))

174

In [57]:
len(cog.filterEdges(lambda e: e.weight > 5))
len(postg.filterEdges(lambda e: e.weight > 1))

34630